# Packages

In [ ]:
import GRN
import GRNCreationUtils
from utils import *
from genesGroup import subgraph3N
import pandas as pd
import networkx as nx
import matplotlib as plt

# Functions

In [ ]:
import networkx as nx
import itertools
from collections import Counter
from networkx.algorithms.isomorphism import DiGraphMatcher

def generate_motif_templates():
    """Returns a dictionary of canonical 3-node motif templates."""
    templates = {}

    def add_motif(name, edges):
        G = nx.DiGraph()
        G.add_edges_from(edges)
        templates[name] = G

    # Basic motif types (based on Alon 2002 classification)
    add_motif("Fan-In", [(1, 0), (2, 0)])
    add_motif("Fan-Out", [(0, 1), (0, 2)])
    add_motif("Cascade", [(0, 1), (1, 2)])
    add_motif("FFL", [(0, 1), (1, 2), (0, 2)])
    add_motif("FBL", [(0, 1), (1, 2), (2, 0)])
    add_motif("Bi-Mutual", [(0, 1), (1, 0), (1, 2), (2, 1)])
    add_motif("Clique", [(0, 1), (1, 0), (0, 2), (2, 0), (1, 2), (2, 1)])
    # Add more motifs if needed

    return templates


def classify_subgraph(subG, motif_templates):
    """Matches a 3-node subgraph to a motif type using isomorphism."""
    for name, motifG in motif_templates.items():
        matcher = DiGraphMatcher(subG, motifG)
        if matcher.is_isomorphic():
            return name
    return "Other"


def count_motifs(G):
    """
    Enumerates and classifies all 3-node subgraphs in a directed graph.
    
    Returns:
        motif_counts (Counter): motif type -> count
    """
    motif_templates = generate_motif_templates()
    motif_counts = Counter()
    
    for nodes in itertools.combinations(G.nodes, 3):
        subG = G.subgraph(nodes).copy()
        if nx.is_weakly_connected(subG):  # Optional: only count connected
            motif_type = classify_subgraph(subG, motif_templates)
            motif_counts[motif_type] += 1

    return motif_counts


def randomize_graph(G, n_swaps=1000, seed=None):
    """
    Returns a randomized version of the graph preserving in/out-degree distribution.
    Uses the Maslov-Sneppen edge-rewiring algorithm.
    """
    G_rand = G.copy()
    nx.double_edge_swap(G_rand, n_swaps=n_swaps, max_tries=n_swaps * 10, seed=seed)
    return G_rand